In [2]:
import numpy as np
from scipy.spatial.distance import cdist
from scipy import ndimage
from scipy import signal
from scipy.stats import linregress
from scipy.signal import convolve2d, gaussian, argrelextrema
from scipy.interpolate import interp1d
from sklearn.preprocessing import normalize
from skimage.transform import rotate
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import pickle as cPickle
import librosa
import glob
import re
import sys
import os
import mido
from mido import MidiFile
import cv2
from PIL import Image

# Strip Analysis Functions

### Load Strips

In [3]:
pkl_dir = 'line_pred_pkl_data'

In [4]:
def sort_nicely(l):
    d = {}
    for elem in l:
        m = re.search('\S+-(\d+)\.pkl', elem)
        if m:
            idx = int(m.group(1))
            d[idx] = elem
        else:
            print('Failed regular expression matching: %s' % (elem))
            sys.exit(1)
    sorted_filenames = [d[key] for key in sorted(d.keys())]
    return sorted_filenames

In [29]:
def loadStripData(basename):
    pkl_files = sort_nicely(glob.glob(pkl_dir + '/' + basename + '-*.pkl'))
    # debugging for loop
    for pkl_file in pkl_files:
        print(pkl_file)
    strips = []
    for pkl_file in pkl_files:
        with open(pkl_file, 'rb') as f:
            d = cPickle.load(f)
            print("length of d: " + str(len(d)))
            strips.extend(d['strips'])
    print("length of strips: " + str(len(strips)))
    return strips

In [6]:
def showImage(X, sz = (12,12)):
    plt.figure(figsize=sz)
    plt.imshow(1-X, cmap='gray')

In [7]:
def showStrips(strips):
    for strip in strips:
        showImage(strip)

In [19]:
scoreFile = 'childrencorner6_vmusescore'

In [30]:
strips = loadStripData(scoreFile)

line_pred_pkl_data/childrencorner6_vmusescore-0.pkl
line_pred_pkl_data/childrencorner6_vmusescore-1.pkl
line_pred_pkl_data/childrencorner6_vmusescore-2.pkl
line_pred_pkl_data/childrencorner6_vmusescore-3.pkl
line_pred_pkl_data/childrencorner6_vmusescore-4.pkl
length of strips: 0


In [26]:
file = 'line_pred_pkl_data/childrencorner6_vmusescore-0.pkl'
with open(file, 'rb') as f:
    d = cPickle.load(f)

In [27]:
len(strips)

0

In [10]:
showImage(strips[0])

IndexError: list index out of range

### Strip Analysis

In [2]:
def locateStaffLines(s, delta = 3):
    medvals = np.median(s, axis=1)
    for i, curVal in enumerate(medvals):
        if i > delta and i < len(medvals)-delta:
            if curVal != np.max(medvals[i-delta:i+delta+1]): # only keep local maxima
                medvals[i] = 0
    idxs_sorted_val = np.argsort(medvals)[::-1]
    idxs_lines = sorted(idxs_sorted_val[0:10])
    return np.array(idxs_lines)

In [3]:
def estimateMidiNum2PixelRowMapping(s, lineIdxs, hand = 'right'):
    if hand == 'right':
        x = np.array([77, 74, 71, 67, 64]) # midi nums for staff lines
    elif hand == 'left':
        x = np.array([57, 53, 50, 47, 43])
    else:
        print('Unrecognized value for hand: %s' % hand)
        sys.exit(1)
    m, b, _, _, _ = linregress(x, lineIdxs)
    return (m,b)

In [4]:
def estimatePixelRow(midinum, m, b):
    return np.int(np.round(midinum * m + b))